In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.1.1


In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.1.1


In [2]:
import pickle
import pandas as pd
import numpy as np
import argparse

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

/home/rooster/Documentos/mlops/mlops-zoomcamp/04-deployment/env/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/rooster/Documentos/mlops/mlops-zoomcamp/04-deployment/env/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [28]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-04.parquet')

In [29]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)
np.mean(y_pred)

16.551162930515858

In [19]:
y_pred.shape, df.shape

((990113,), (990113, 9))

In [15]:
year  = 2021
month = 2

In [16]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [20]:
df['predictions'] = y_pred

In [21]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration,ride_id,predictions
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173,82,NaN,B00021,10.666667,2021/02_1,14.539865
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173,56,NaN,B00021,14.566667,2021/02_2,13.740422
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82,129,NaN,B00021,7.950000,2021/02_3,15.593339
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,-1,225,NaN,B00037,13.800000,2021/02_4,15.188118
5,B00037,2021-02-01 00:00:37,2021-02-01 00:09:35,-1,61,NaN,B00037,8.966667,2021/02_5,13.817206


In [24]:
df.to_parquet(
    path="predict.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)

In [3]:
parser    = argparse.ArgumentParser()
subparser = parser.add_subparsers(dest='command')

In [4]:
params = subparser.add_parser('')
params.add_argument('--year', type=int, required=True)
params.add_argument('--month', type=int, required=True)

_StoreAction(option_strings=['--month'], dest='month', nargs=None, const=None, default=None, type=<class 'int'>, choices=None, help=None, metavar=None)